In [14]:
import os
import math
import random
import argparse
from collections import Counter, defaultdict
from itertools import chain, combinations
import ast
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression


### Local Imports

In [15]:
from utils.Spambase.split_data import split_data_equal
from utils.aggregate_functions import aggregate_lr_models
from utils.evaluate_coalitions import evaluate_coalitions
from utils.nash1 import find_nash_equilibria_v2
from utils.evaluate_coalitions_new import evaluate_coalitions2

In [16]:
from utils.nash_lot import find_nash_equilibria_lottery

In [17]:
# disable ConvergenceWarnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# HuGaDB Dataset

In [ ]:
def payoff_(mu: float, sigma: float) -> float:
    return         0.1106 + 0.8822 * mu + 0.7949 * sigma

def payoff_100(mu: float, sigma: float) -> float:
    return         -0.0102 + 1.0908 * mu + 0.7913 * sigma

def payoff_lr(mu: float, sigma: float) -> float:
    return         + 0.0681 + 0.9307 * mu + 1.4150 * sigma

In [23]:
# File patterns.
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/train_{i:02d}.csv"
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/test_{i:02d}.csv"

# Concatenate all training files.
df_train_global = pd.concat([
    pd.read_csv(train_files_pattern.format(i=i)) for i in range(1, 11)
]).dropna()

# Concatenate all testing files.
df_test_global = pd.concat([
    pd.read_csv(test_files_pattern.format(i=i)) for i in range(1, 11)
]).dropna()

# Split features and labels.
X_train_global = df_train_global.drop('act', axis=1)
y_train_global = df_train_global['act']

X_test_global = df_test_global.drop('act', axis=1)
y_test_global = df_test_global['act']

# Encode labels.
label_encoder = LabelEncoder()
y_train_global = label_encoder.fit_transform(y_train_global)
y_test_global = label_encoder.transform(y_test_global)

# Scale features.
scaler_global = StandardScaler()
X_train_global_scaled = scaler_global.fit_transform(X_train_global)
X_test_global_scaled  = scaler_global.transform(X_test_global)

In [24]:
import numpy as np
import joblib

# 1) load the pipeline
pipeline = joblib.load('/Users/abbaszal/Documents/global_acc_pipeline.joblib')


In [25]:
# Parameters
eps               = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials          = 100
base_random_seed  = 42
max_iters         = [ 100]
approach          = 'fedlr'

# Saving directory
save_dir = "/Users/abbaszal/Documents/thimasfit11/hugadb_fedlr_all_joblib"
os.makedirs(save_dir, exist_ok=True)

train_csv_path = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/new_runs/train.csv"
df_full_train = pd.read_csv(train_csv_path).dropna(subset=['act']).reset_index(drop=True)

for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]
    #print(payoff_func)
    
    for max_iter in max_iters:
        print(f"  max_iter = {max_iter}")

        # reset counters for this config
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials + 1):
            
            rand_component = random.randint(0, 500)
            trial_seed = base_random_seed + trial + int(1000 * max_iter) + 2 * rand_component


            X_test_full_scaled = X_test_global_scaled.copy()
            y_test_full        = y_test_global.copy()

            # Create a stratified subsample of the test set to speed up the runtime.
            subsample_size = 950  
            X_test_glob, _, y_test_glob, _ = train_test_split(
                X_test_full_scaled, y_test_full,
                train_size=subsample_size,
                random_state=trial_seed,
                stratify=y_test_full
            )


            df_trial, _ = train_test_split(
                df_full_train,
                train_size=10000,
                random_state=trial_seed,
                stratify=df_full_train['act']
            )
   
            df_remaining = df_trial.copy()


            client_partitions = []

            sample_size = 5000 // n_clients
            for client_idx in range(n_clients):
                if len(df_remaining) < sample_size:
                    raise ValueError("err")

                if len(df_remaining) == sample_size:
                    df_client = df_remaining.copy()
                    df_remaining = df_remaining.iloc[0:0] 

                else:
                    df_client, df_remaining = train_test_split(
                        df_remaining,
                        train_size=sample_size,
                        random_state=trial_seed,
                        stratify=df_remaining['act']
                    )
                df_client = df_client.reset_index(drop=True)
                client_partitions.append(df_client)

            # train each client
            client_models = []
            client_accs   = {}

            for client_idx, df_client in enumerate(client_partitions):

                X_client = df_client.drop(columns=['act'])
                y_client = df_client['act']

                client_scaler = StandardScaler()
                X_client_scaled = client_scaler.fit_transform(X_client)
                y_client_encoded = label_encoder.transform(y_client)

                mdl = LogisticRegression(random_state=trial_seed,
                                         max_iter=max_iter)
                mdl.fit(X_client_scaled, y_client_encoded)
                client_models.append(mdl)

                pred = mdl.predict(X_test_glob)
                client_accs[client_idx] = accuracy_score(y_test_glob, pred)

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=aggregate_lr_models,
                X_test=X_test_glob,
                y_test=y_test_glob,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals      = np.array(list(client_accs.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)

            # 3) package into a 2-column array and predict
            X_new    = np.array([[mu_full, sig_full]])   # shape (1,2)
            payoff_f    = pipeline.predict(X_new)[0]
            print(payoff_f)
            #payoff_f  = payoff_100(mu_full, sig_full)
           # print(payoff_f)

            has_incentive = any(
                acc > payoff_f + eps
                for acc in client_accs.values()
            )
            if not has_incentive:
                lottery_count += 1


        static_count = sum(counts_static.values())

        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_iter,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                 f"_nclients_{n_clients}"
                 f"_maxiter_{max_iter}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f"saved {fname}")



> n_clients = 10
  max_iter = 100
0.671809655531959
0.6645185837721502
0.6608219080578378
0.6769472349534974
0.6620776476450774
0.6912254298496534
0.6722157895299518
0.6554284214212486
0.6549747379478703
0.6590316381845456
0.6826657901354009
0.6729273416234036
0.6726970486563397
0.6573218871136588
0.682717254272728
0.647671707557927
0.6607289952750636
0.6773567354573732
0.6672083420724346
0.6463011477583254
0.656798278805936
0.6620662912157363
0.6586855737863024
0.6729273416234036
0.6638243697528489
0.6613183067249849
0.6693826860699883
0.6697843732939153
0.6615111855088822
0.6770243076563677
0.6634117123892493
0.6627070639466681
0.6655091693888686
0.6780074837002974
0.6505657579620382
0.6722493760074122
0.6814109613527825
0.6491089034142691
0.6814109613527825
0.6601281155658445
0.6585513044106188
0.6720329785897495
0.6504077183706697
0.669895715483423
0.6730405196539204
0.6604793989171812
0.6584208448117623
0.6435560012399042
0.6593498094331777
0.657246233290032
0.6434831450161279
0.

# Spambase Dataset

In [18]:
file_path = '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/spambase.data'  # Adjust the path as needed
df = pd.read_csv(file_path, header=None)

In [6]:
def payoff_(mu: float, sigma: float) -> float:
    return 0.5547 + 0.4137 * mu -0.0490 * sigma

def payoff_100(mu: float, sigma: float) -> float:
    return 0.5879 + 0.3562 * mu + 0.1891 * sigma

In [19]:
def train_models_fedlr(partitions, random_seed, X_test, y_test, max_iter):
    client_models = []
    client_global_accuracies = []
    
    for X_i, y_i in partitions:
        nan_mask = ~np.isnan(X_i).any(axis=1)
        X_clean = X_i[nan_mask]
        y_clean = y_i[nan_mask]
        if len(y_clean) == 0:
            client_models.append(None)
            client_global_accuracies.append(None)
            continue
        
        model = LogisticRegression(random_state=random_seed, max_iter=max_iter)
        try:
            local_scaler = StandardScaler()
            model.fit(local_scaler.fit_transform(X_clean), y_clean)
            client_models.append(model)
            client_global_accuracies.append(model.score(X_test, y_test))
        except Exception as e:
            client_models.append(None)
            client_global_accuracies.append(None)
    
    return client_models, client_global_accuracies

In [20]:
import numpy as np
import joblib

# 1) load the pipeline
pipeline = joblib.load('/Users/abbaszal/Documents/global_acc_pipeline.joblib')


In [22]:
# Parameters
eps     = 1e-8
n_trials    = 100
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
base_seed   = 42
max_iters   = [100]
approach    = 'fedlr'


save_dir = (
"/Users/abbaszal/Documents/thimasfit11/spambase_fedlr_all_joblib"
)
os.makedirs(save_dir, exist_ok=True)

all_results = []


for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]

    for max_iter in max_iters:
        print(f"\n max_iter = {max_iter} ")
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials+1):
            rc = random.randint(0, 500)
            trial_seed = base_seed + (trial-1) + 1000*max_iter + 2*rc
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            X = df.iloc[:, :-1].to_numpy()
            y = df.iloc[:, -1].to_numpy()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial_seed)

            scaler = StandardScaler()

            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            partitions = split_data_equal(
                X_train, y_train,
                n_clients=n_clients,
                shuffle=True,
                random_seed=trial_seed
            )
            
            client_models, client_accs = train_models_fedlr(
                partitions=partitions,
                random_seed=trial_seed,
                X_test=X_test,
                y_test=y_test,
                max_iter=max_iter
            )

    
            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=aggregate_lr_models,
                X_test=X_test,
                y_test=y_test,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            # 2) compute your new μ, σ
            vals = np.array(client_accs)
            mu_full  = vals.mean()
            sig_full = vals.std(ddof=1)

            # 3) package into a 2-column array and predict
            X_new    = np.array([[mu_full, sig_full]])   # shape (1,2)
            payoff_f    = pipeline.predict(X_new)[0]
            print(payoff_f)



            has_incentive = any(
                acc > payoff_f 
                for acc in client_accs
            )
            if not has_incentive:
                lottery_count += 1

        # aggregate counts into single numbers
        static_count = sum(counts_static.values())


        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_iter,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                    f"_nclients_{n_clients}"
                    f"_maxiter_{max_iter}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f" saved {fname}")


> n_clients = 10

 max_iter = 100 
0.9189775480909621
0.9306188285947452
0.9251708870087252
0.9312127734259694
0.9240475183149184
0.9238371686133954
0.908845648682492
0.9108268877305591
0.9180225388178124
0.9108268877305591
0.9132330813273445
0.9184396827150683
0.927350647792154
0.9230586540433044
0.9289802397011001
0.9051912086736265
0.9233578692817362
0.9283285995304427
0.922954937859703
0.9175426976033193
0.9273556296228783
0.9089290486835543
0.9180142296202639
0.9215199405674654
0.9282480839958688
0.9260756242771334
0.9160465890347695
0.9107941072256712
0.9203797188091885
0.9195240987527937
0.906656746663025
0.9238613605371536
0.9120962042667524
0.9263926038301826
0.9074619936160503
0.9266410399492016
0.9186336967029132
0.9194034102020336
0.9232848322096555
0.9225494027695539
0.925343699559932
0.9089921823862197
0.9288209771699507
0.9213254690813268
0.918073055845442
0.9252568519524496
0.926234436104028
0.9167553335169599
0.9193219117714297
0.9272708869760512
0.9245629140054594
0.

In [ ]:
# Parameters
eps     = 1e-8
n_trials    = 100
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
base_seed   = 42
max_iters   = [100]
approach    = 'fedlr'


save_dir = (
"/Users/abbaszal/Documents/thimasfit11/spambase_fedlr_all_test"
)
os.makedirs(save_dir, exist_ok=True)

all_results = []


for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]

    for max_iter in max_iters:
        print(f"\n max_iter = {max_iter} ")
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials+1):
            rc = random.randint(0, 500)
            trial_seed = base_seed + (trial-1) + 1000*max_iter + 2*rc
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            X = df.iloc[:, :-1].to_numpy()
            y = df.iloc[:, -1].to_numpy()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial_seed)

            scaler = StandardScaler()

            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            partitions = split_data_equal(
                X_train, y_train,
                n_clients=n_clients,
                shuffle=True,
                random_seed=trial_seed
            )
            
            client_models, client_accs = train_models_fedlr(
                partitions=partitions,
                random_seed=trial_seed,
                X_test=X_test,
                y_test=y_test,
                max_iter=max_iter
            )

    
            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=aggregate_lr_models,
                X_test=X_test,
                y_test=y_test,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals = np.array(client_accs)
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)
            payoff_f  = payoff_100(mu_full, sig_full)
            print(payoff_f)



            has_incentive = any(
                acc > payoff_f 
                for acc in client_accs
            )
            if not has_incentive:
                lottery_count += 1

        # aggregate counts into single numbers
        static_count = sum(counts_static.values())


        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_iter,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                    f"_nclients_{n_clients}"
                    f"_maxiter_{max_iter}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f" saved {fname}")

In [ ]:
import pandas as pd
import os

# Load the data
file_path = "/Users/abbaszal/Documents/Spambase_results_with_LR.csv"  # Adjust path if needed
df = pd.read_csv(file_path)

# Identify columns containing client accuracies
client_accuracy_cols = [col for col in df.columns if "Client" in col and "Accuracy" in col]

# Also keep MaxIter and Trial columns
columns_to_keep = ['MaxIter', 'Trial'] + client_accuracy_cols

# Get first row for each (MaxIter, Trial)
df_first_rows = df.groupby(['MaxIter', 'Trial'], as_index=False).first()

# Select only the desired columns
df_filtered = df_first_rows[columns_to_keep]

# Save the filtered dataframe
save_path = "/Users/abbaszal/Documents/Spambase_FedLR_Fit.csv"
df_filtered.to_csv(save_path, index=False)

print(f"Saved filtered client accuracies to {save_path}")

Saved filtered client accuracies to /Users/abbaszal/Documents/Spambase_FedLR_Fit.csv
